## Libraries

In [5]:
import pandas as pd
import numpy as np
import json
import os
import csv
from tqdm import tqdm

## Functions

### Flattener function

In [6]:
def flatten_nested_df(df):
    list_error = []
    dict_error = []

    try:
        s = (df.applymap(type) == list).any()
        list_columns = s[s].index.tolist()

        s = (df.applymap(type) == dict).any()
        dict_columns = s[s].index.tolist()

        while (len(list_columns) > 0 or len(dict_columns) > 0) and ((set(list_columns) != set(list_error)) or (set(dict_columns) != set(dict_error))):
            for x in list_columns:
                try:
                    df = df.explode(x, ignore_index=False)
                    df = df.reset_index(drop=True)
                except:
                    print(f"Error en: {x}")
                    list_error.append(x)
                    pass

            for y in dict_columns:
                try:
                    df_new = pd.json_normalize(df[y].dropna()).add_prefix(f'{y}.')
                    df = df.merge(df_new, how='left', left_index=True, right_index=True)
                    del(df[y])
                except:
                    print(f"Error en: {y}")
                    dict_error.append(y)
                    pass

            s = (df.applymap(type) == list).any()
            list_columns = s[s].index.tolist()

            s = (df.applymap(type) == dict).any()
            dict_columns = s[s].index.tolist()
        
        return df
    except:
        return df




### Verify flattening

In [7]:
def verify_flattening(df):
    s = (df.applymap(type) == list).any()
    list_columns = s[s].index.tolist()
    
    s = (df.applymap(type) == dict).any()
    dict_columns = s[s].index.tolist()

    print(f"Listas: {list_columns}, Diccionarios: {dict_columns}")

### Remove unwanted columns

In [8]:
def remove_columns_df(df, list_columns):
    columns_to_drop = [col for col in df.columns if col not in list_columns]
    df.drop(columns = columns_to_drop, inplace = True)
    df.drop_duplicates(inplace = True)
    
    return df    

### Iteration pipeline

In [9]:
def smasher_filtered(json_data, list_columns):
    stage_df = pd.DataFrame()
    total_documents = len(json_data)

    with tqdm(total = total_documents) as pbar:
        for x in range(0, total_documents):
            j_norm = pd.json_normalize(json_data[x])
            j_flat = flatten_nested_df(j_norm)
            stage_df = pd.concat([remove_columns_df(j_flat), stage_df])
            pbar.update(1)
            
    print(f"Completed. Final dataframe shape: {stage_df.shape}")
    return stage_df

In [10]:
def smasher(json_data):
    stage_df = pd.DataFrame()
    total_documents = len(json_data)

    with tqdm(total = total_documents) as pbar:
        for x in range(0, total_documents):
            j_norm = pd.json_normalize(json_data[x])
            j_flat = flatten_nested_df(j_norm)
            stage_df = pd.concat([j_flat, stage_df])
            pbar.update(1)
        
    print(f"Completed. Final dataframe shape: {stage_df.shape}")
    return stage_df

## Parameters

In [21]:
PARAM = {}

# Working directory
PARAM['working_directory'] = "C:/Users/ferna/OneDrive - Universidad Austral/Git/json-smasher"

# Input file name
PARAM['input_file_name'] = "large-file.json"

# Output file name
PARAM['output_file_name'] = "large-file"
PARAM['output_excel'] = True

# Filters
PARAM['col_filter'] = False
columns_filter = []

PARAM['row_filter'] = False
rows_filter = range(0,10)

## Read json file

In [11]:
with open('large-file.json', 'r', encoding="utf8") as f:
    json_data = json.load(f)

## Filter (Optional)

In [9]:
if PARAM['col_filter'] and PARAM['row_filter']:
    filtered_rows = json_data[rows_filter]
    filtered_json = [{k:v for k,v in d.items() if k in columns_filter} for d in json_data]
elif PARAM['col_filter']:
    filtered_json = [{k:v for k,v in d.items() if k in columns_filter} for d in json_data]
elif PARAM['row_filter']:
    filtered_json = json_data[rows_filter]
else:
    filtered_json = json_data

## Execute

In [12]:
middle_flat = smasher(json_data)

  0%|          | 0/11351 [00:00<?, ?it/s]

100%|██████████| 11351/11351 [42:34<00:00,  4.44it/s]

Completed. Final dataframe shape: (15931, 657)


## Remove duplicated records

In [13]:
initial_length = len(middle_flat)
final_flat = middle_flat.drop_duplicates()
final_length = len(final_flat)

if (initial_length == final_length):
    print('No duplicates found')
else:
    print( f'Initial length was: {initial_length}, and after removing duplicates is: {final_length}')

No duplicates found


## Save to file

In [34]:
if PARAM['output_excel']:
    xlsx_path = PARAM['working_directory']+'/'+PARAM['output_file_name']+'.xlsx'
    #final_flat.to_excel(xlsx_path, index=False)
    writer = pd.ExcelWriter(xlsx_path, engine="xlsxwriter", options={'strings_to_urls': False})
    final_flat.to_excel(writer)
    writer.save()
else:
    csv_path = os.path.join(PARAM['working_directory'], PARAM['output_file_name']+'.csv')
    final_flat.to_csv(csv_path, sep=',', index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

C:\Users\ferna\AppData\Local\Temp\ipykernel_11924\2293286817.py:4: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(xlsx_path, engine="xlsxwriter", options={'strings_to_urls': False})
c:\Users\ferna\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [19]:
final_flat

,id,type,public,created_at,actor.id,actor.login,actor.gravatar_id,actor.url,actor.avatar_url,repo.id,...,payload.release.assets.uploader.gists_url,payload.release.assets.uploader.starred_url,payload.release.assets.uploader.subscriptions_url,payload.release.assets.uploader.organizations_url,payload.release.assets.uploader.repos_url,payload.release.assets.uploader.events_url,payload.release.assets.uploader.received_events_url,payload.release.assets.uploader.type,payload.release.assets.uploader.site_admin,payload.pull_request.head.repo
0,2489678844,IssuesEvent,True,2015-01-01T15:59:59Z,9343331,No-CQRT,,https://api.github.com/users/No-CQRT,https://avatars.githubusercontent.com/u/9343331?,25600089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2489678843,IssuesEvent,True,2015-01-01T15:59:59Z,9343331,No-CQRT,,https://api.github.com/users/No-CQRT,https://avatars.githubusercontent.com/u/9343331?,25600089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2489678842,DeleteEvent,True,2015-01-01T15:59:59Z,2057932,LipkeGu,,https://api.github.com/users/LipkeGu,https://avatars.githubusercontent.com/u/2057932?,24847217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2489678840,IssuesEvent,True,2015-01-01T15:59:59Z,9343331,No-CQRT,,https://api.github.com/users/No-CQRT,https://avatars.githubusercontent.com/u/9343331?,25600089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2489678838,IssuesEvent,True,2015-01-01T15:59:59Z,1105372,ophian,,https://api.github.com/users/ophian,https://avatars.githubusercontent.com/u/1105372?,2627116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2489651062,PushEvent,True,2015-01-01T15:00:03Z,485033,winterbe,,https://api.github.com/users/winterbe,https://avatars.githubusercontent.com/u/485033?,28593843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2489651057,WatchEvent,True,2015-01-01T15:00:03Z,6894991,SametSisartenep,,https://api.github.com/users/SametSisartenep,https://avatars.githubusercontent.com/u/6894991?,2871998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2489651053,PushEvent,True,2015-01-01T15:00:01Z,6339799,izuzero,,https://api.github.com/users/izuzero,https://avatars.githubusercontent.com/u/6339799?,28270952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2489651051,PushEvent,True,2015-01-01T15:00:01Z,3854017,rspt,,https://api.github.com/users/rspt,https://avatars.githubusercontent.com/u/3854017?,28671719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
